In [2]:
# get the environment
import sys
print(sys.version)
print(sys.executable)

3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
C:\Users\bbutler\Documents\Anaconda3\envs\timeseries\python.exe


In [3]:
# import base libraries
from IPython.core.display import display, HTML
from dateutil import relativedelta
import pandas as pd
import numpy as np
import pyodbc
import os
import re
import statistics as stat
import datetime as dt
from datetime import timedelta

# additional imports for visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# set options
display(HTML("<style>.container {width:90% !important;} </style>"))
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.2f}'.format

In [4]:
def AP2_QUERY(sql):
    con = pyodbc.connect('DSN=AP2')
    cursor = con.cursor()
    c = cursor.execute(sql)
    dff = pd.DataFrame.from_records(c.fetchall(),columns=[x[0] for x in c.description])
    return dff
    cursor.close()
    con.close() 


In [26]:
check = AP2_QUERY('''
    SELECT TOP 10 * FROM Bank_Customer.cv_Bank_Customer_To_Account_Relationships C2A WHERE 
    Account_Class_Name LIKE 'LOANS' AND 
    Prospect_Code LIKE 'C' AND 
    Customer_Type LIKE 'Individual' AND 
    Customer_Status_Code LIKE 'O'
''')

check

,As_of_Date,Application_Code,Cust_CIS_Nbr,Cust_CIS_Nbr_String,Account_Number,Account_Group_Number,Account_Class_Name,Customer_Type_Code,Customer_Type,Relation_Type_Code,Relation_Type_Descr,Role_Code,Role_Descr,Owner_Ind,Customer_Name_Line,Gender_Code,Prospect_Code,Customer_Status_Code,Customer_Tax_Code
0,2020-08-10,LN,2406916,00002406916,03750019255-00001,03750019255,LOANS,I,Individual,901,PRIMARY,000,SOLE OWNER,O,SCOTT D COLAMETA,,C,O,S
1,2020-08-10,LN,2406976,00002406976,03750019629-00001,03750019629,LOANS,I,Individual,901,PRIMARY,000,SOLE OWNER,O,MARYLOU MONTISANO,,C,O,S
2,2020-08-10,LNCMT,2406981,00002406981,03750019385-100,03750019385,LOANS,I,Individual,901,PRIMARY,022,JOINT OWNER,O,MICHAEL HICKS,,C,O,S
3,2020-08-10,LNCMT,2406981,00002406981,03750019385-001,03750019385,LOANS,I,Individual,901,PRIMARY,022,JOINT OWNER,O,MICHAEL HICKS,,C,O,S
4,2020-08-10,LN,2406981,00002406981,03750019385-00001,03750019385,LOANS,I,Individual,901,PRIMARY,022,JOINT OWNER,O,MICHAEL HICKS,,C,O,S
5,2020-08-10,LNCMT,2406999,00002406999,03750019389-001,03750019389,LOANS,I,Individual,901,PRIMARY,022,JOINT OWNER,O,ROBERT A HAGAN,,C,O,S
6,2020-08-10,LNCMT,2406999,00002406999,03750019389-100,03750019389,LOANS,I,Individual,901,PRIMARY,022,JOINT OWNER,O,ROBERT A HAGAN,,C,O,S
7,2020-08-10,LN,2406999,00002406999,03750019389-00001,03750019389,LOANS,I,Individual,901,PRIMARY,022,JOINT OWNER,O,ROBERT A HAGAN,,C,O,S
8,2020-08-10,LN,2407013,00002407013,03750019477-00001,03750019477,LOANS,I,Individual,901,PRIMARY,000,SOLE OWNER,O,DENNIS A REDDY,,C,O,S
9,2020-08-10,LN,2407016,00002407016,03750019340-00001,03750019340,LOANS,I,Individual,901,PRIMARY,022,JOINT OWNER,O,DENNIS CLOSE,M,C,O,S


In [23]:
res = AP2_QUERY('''
    SELECT TOP 10 Cust_CIS_Nbr, Account_Number, Account_Group_Number, Role_Descr
    FROM Bank_Customer.cv_Bank_Customer_To_Account_Relationships C2A WHERE 
    Account_Class_Name LIKE 'LOANS' AND 
    Prospect_Code LIKE 'C' AND 
    Customer_Type LIKE 'Individual' AND 
    Customer_Status_Code LIKE 'O'
''')

In [24]:
res

,Cust_CIS_Nbr,Account_Group_Number,Role_Descr
0,2406916,03750019255,SOLE OWNER
1,2406976,03750019629,SOLE OWNER
2,2406981,03750019385,JOINT OWNER
3,2406981,03750019385,JOINT OWNER
4,2406981,03750019385,JOINT OWNER
5,2406999,03750019389,JOINT OWNER
6,2406999,03750019389,JOINT OWNER
7,2406999,03750019389,JOINT OWNER
8,2407013,03750019477,SOLE OWNER
9,2407016,03750019340,JOINT OWNER


In [13]:
res2 = AP2_QUERY('''
    SELECT TOP 10 * FROM Bank_Customer.cv_Bank_Customer_Address CA
''')

res2

,As_Of_Date,Cust_CIS_Nbr,Customer_Type,Customer_Full_Name,Customer_First_Name,Customer_Middle_Name,Customer_Last_Name,Customer_First_Middle_Name,Customer_Primary_Phone_Number,Customer_Secondary_Phone_Number,Address_Mailing_Info,Address_Street,Address_City,Address_State_Code,Address_Zip_Code,Address_City_State_Zip,Address_Country_Code,Address_Country_Name,Effective_Date,Name_Last_Change_Date,Address_Last_Change_Date
0,2020-08-10,00002358068,I,JUSTYN D BAXTER,JUSTYN,D,BAXTER,JUSTYN D,5089272733,None,None,44 SEABROOK RD,EAST FALMOUTH,MA,02536,EAST FALMOUTH MA 02536,USA,UNITED STATES OF AMERICA,2019-04-22,2019-04-22,2019-01-28
1,2020-08-10,00000985072,I,WILLIAM HENRY RUSSELL,WILLIAM,HENRY,RUSSELL,WILLIAM HENRY,None,None,None,308 NAHANT RD,NAHANT,MA,019081476,NAHANT MA 01908-1476,USA,UNITED STATES OF AMERICA,2009-06-09,2009-06-09,2008-12-30
2,2020-08-10,00000985048,I,LILLIANA F MAGANZINI,LILLIANA,F,MAGANZINI,LILLIANA F,7813915812,None,None,20 MORGAN AVE,MEDFORD,MA,021551020,MEDFORD MA 02155-1020,USA,UNITED STATES OF AMERICA,2009-06-09,2009-06-09,2008-12-30
3,2020-08-10,00001913968,I,THALYA N PHAM,THALYA,N,PHAM,THALYA N,6176940836,6175587611,None,62 WALKER ST,QUINCY,MA,02171,QUINCY MA 02171,USA,UNITED STATES OF AMERICA,2013-11-21,2013-11-21,2012-05-31
4,2020-08-10,00000985398,I,DAVID T ANDERSON,DAVID,T,ANDERSON,DAVID T,7816314856,6177233600,None,21 LEE ST,MARBLEHEAD,MA,019453214,MARBLEHEAD MA 01945-3214,USA,UNITED STATES OF AMERICA,2013-06-12,2013-06-12,2008-12-30
5,2020-08-10,00002358092,O,POST HOLDINGS LLC,HOLDINGS,LLC,None,None,None,None,None,6 HARBOUR TOWN POINT,SHOAL CREEK,AL,35242,SHOAL CREEK AL 35242,USA,UNITED STATES OF AMERICA,2019-01-31,2019-01-31,2018-06-25
6,2020-08-10,00001650401,I,MARINA NIKITINA,MARINA,None,NIKITINA,MARINA,7815958927,None,None,21 CHERRY ST,LYNN,MA,01902,LYNN MA 01902,USA,UNITED STATES OF AMERICA,2015-08-02,2015-08-02,2010-08-04
7,2020-08-10,00002358033,I,BRINELGY CONTRERAS MARTINEZ,BRINELGY,CONTR,MARTINEZ,BRINELGY CONTRERAS,7815882320,None,None,18 MAVERICK ST APT 1,CHELSEA,MA,02150,CHELSEA MA 02150,USA,UNITED STATES OF AMERICA,2019-01-28,2019-01-28,2019-01-28
8,2020-08-10,00002357969,I,STEVEN MEI,STEVEN,None,MEI,STEVEN,None,None,None,None,None,None,None,None,None,None,2019-01-28,2019-01-28,1996-09-11
9,2020-08-10,00000985727,I,DONALD CRAVEN JR,DONALD,None,CRAVEN,DONALD,6178402747,None,None,7 BRATTLE DR,ARLINGTON,MA,024742802,ARLINGTON MA 02474-2802,USA,UNITED STATES OF AMERICA,2009-06-09,2009-06-09,2007-03-07


In [6]:
res2 = AP2_QUERY('''
    SELECT TOP 10 * FROM Loan.cv_Bank_Loan_Note_Info_Monthly
''')

res2

,Month_End_Date,Note_Account_Number,Note_Acct_Nbr_No_Hyphen,Loan_Account_Number,Note_Number,DAY_TIME_ID,Year_Month_Number,Note_Type_Code,Note_Type_Descr,Paid_Code,Paid_Status,Paid_Date,Paid_off_This_Month_Ind,Entered_Date,Entered_This_Month_Ind,Note_Cost_Center_Number,Note_Branch_Number,Expected_Maturity_Date,Issue_Date,Last_Payment_Date,Renewal_Date,Maturity_Date,Current_Interest_Rate,Original_Interest_Rate,Accrual_Ind,Annual_Percentage_Rate,Note_Bankrupt_Code,Charge_Off_Code,Charge_Off_Effective_Date,Past_Due_Ind,Past_Due_Balance_Range_Code,Delinquency_Reason_Code,Delinquency_Reason_Date,Days_Late_15_Ind,Days_Late_30_Ind,Days_Late_60_Ind,Days_Late_90_Ind,Days_Late_120_Ind,Account_Number_CNT,SUM_Ledger_Balance_Amount,SUM_Note_15_DAY_Past_CNT,SUM_Note_30_DAY_Past_CNT,SUM_Note_60_DAY_Past_CNT,SUM_Note_90_DAY_Past_CNT,SUM_Note_120_DAY_Past_CNT,SUM_Day_Payment_Past_Due_CNT,SUM_Number_Of_Days_Past_Due_CNT,SUM_Past_Due_Amount,SUM_Note_Aggregate_Current_Balance,AVG_Note_Aggregate_Interest_Rate,SUM_Average_Balance_This_Period,SUM_Current_Balance,SUM_Original_Balance
0,2012-06-29,00560409743-00001,0056040974300001,00560409743,1,59350,201206,102,RL CASH RESERVE-VARI,0,Open,0001-01-01,N,1969-02-01,N,375,322,0001-01-01,1969-02-01,2006-07-11,0001-01-01,0001-01-01,11.250000,17.500000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,1,0,0,0,0,0,0,0.00,0.00,348.750000,0.00,0.00,0.00
1,2012-06-29,02300019463-00001,0230001946300001,02300019463,1,59350,201206,100,RL PERS CASH RESERVE,0,Open,0001-01-01,N,1970-01-01,N,375,375,0001-01-01,1970-01-01,None,0001-01-01,0001-01-01,18.000000,18.000000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,0,0,0,0,0,0,0,0.00,0.00,558.000000,0.00,0.00,0.00
2,2012-06-29,02403012428-00001,0240301242800001,02403012428,1,59350,201206,100,RL PERS CASH RESERVE,0,Open,0001-01-01,N,1970-01-01,N,375,375,0001-01-01,1970-01-01,None,0001-01-01,0001-01-01,18.000000,18.000000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,0,0,0,0,0,0,0,0.00,0.00,558.000000,0.00,0.00,0.00
3,2012-06-29,00191000140-00001,0019100014000001,00191000140,1,59350,201206,517,CM RES 1-4 FAM PROP,C,Paid,2011-10-03,N,1966-03-03,N,230,230,2013-02-03,1966-03-03,2011-10-03,2007-06-18,2013-02-03,4.250000,7.000000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,10,4,1,0,0,0,0,0.00,0.00,0.000000,0.00,0.00,76000.00
4,2012-06-29,00150036640-00001,0015003664000001,00150036640,1,59350,201206,102,RL CASH RESERVE-VARI,0,Open,0001-01-01,N,1970-02-01,N,375,362,0001-01-01,1970-02-01,2012-01-04,0001-01-01,0001-01-01,7.250000,13.500000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,1,0,0,0,0,0,0,0.00,0.00,224.750000,0.00,0.00,0.00
5,2012-06-29,00150036673-00001,0015003667300001,00150036673,1,59350,201206,102,RL CASH RESERVE-VARI,0,Open,0001-01-01,N,1959-10-01,N,375,322,0001-01-01,1959-10-01,2002-12-18,0001-01-01,0001-01-01,7.250000,13.500000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,0,0,0,0,0,0,0,0.00,0.00,224.750000,0.00,0.00,0.00
6,2012-06-29,02309034892-00001,0230903489200001,02309034892,1,59350,201206,100,RL PERS CASH RESERVE,0,Open,0001-01-01,N,1970-01-01,N,375,375,0001-01-01,1970-01-01,None,0001-01-01,0001-01-01,18.000000,18.000000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,0,0,0,0,0,0,0,0.00,0.00,558.000000,0.00,0.00,0.00
7,2012-06-29,02303123165-00001,0230312316500001,02303123165,1,59350,201206,100,RL PERS CASH RESERVE,0,Open,0001-01-01,N,1970-01-01,N,375,375,0001-01-01,1970-01-01,None,0001-01-01,0001-01-01,18.000000,18.000000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,0,0,0,0,0,0,0,0.00,0.00,558.000000,0.00,0.00,0.00
8,2012-06-29,02404008936-00001,0240400893600001,02404008936,1,59350,201206,100,RL PERS CASH RESERVE,0,Open,0001-01-01,N,1970-01-01,N,375,375,0001-01-01,1970-01-01,None,0001-01-01,0001-01-01,18.000000,18.000000,Y,0.000000,0,0,None,N,,None,0001-01-01,N,N,N,N,N,1,0.00,0,0,0,0,0,0,0,0.00,0.00,558.000000,0.00,0.00,0.00
9,2012-06-29,02409011973-00001,0240901197300001,0240901197